# Introduction.
Hi. So, in this part we are going to create the definition of our request to the nutrition matching service.

Suppose we have a user who only knows how many kilocalories he needs, how many fats, proteins and carbohydrates he needs.

In [12]:
from model.Nutrition import Nutrition
nutrition_data = Nutrition(kcal=2679, protein=150, fat=70, carbs=338)

Plus, we know this user likes to eat 4 times a day. 

In [11]:
meal_count_by_day = 4

Based on these data, we need to make up his caloric intake for each meal, as well as make the layout of desired foods for each meal. 
Since no favorite foods are specified, we will apply the standard approach, in which we will put complex carbohydrates in the first part of the day, together with 

# Prepare meals by timepart

In [13]:
meals = []

So, suppose we set limits between the hours when the user can start eating and the last hour when the user wants to eat. In our example it will between 6 A.M and 8 P.M. For this we will use Linear Optimization

In [24]:
start_of_day = 6
end_of_day = 20

min_distance_between_meals = 2
max_distance_between_meals = 8

In [25]:
from ortools.sat.python import cp_model 
meals_hours = []

model = cp_model.CpModel()
for i in range(1, meal_count_by_day):
    meal_model = model.NewIntVar(start_of_day, end_of_day, 'meal_'+str(i+1))
    if len(meals_hours) == 0:
        meal_zero = model.NewIntVar(start_of_day, end_of_day, 'meal_'+str((i)))
        meals_hours.append(meal_zero)
    meal_zero = meals_hours[i-1]
    meals_hours.append(meal_model)
    model.Add(meal_zero < meal_model)    
    model.Add(meal_model - meal_zero >= min_distance_between_meals)
    model.Add(meal_model - meal_zero <= max_distance_between_meals)

In [26]:

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for resolved_time in meals_hours:
        print(f"{resolved_time}: {solver.Value(resolved_time)} ч.")
else: 
    print("ERROR NOT FOUND")        

meal_1: 6 ч.
meal_2: 10 ч.
meal_3: 16 ч.
meal_4: 20 ч.
